In [1]:
from setup_general import *
from setup_embedding import *

# Setup


In [2]:
type_indicators = {}
with open('data/type_indicators/type_ind_cut.txt', 'r') as f:
    for line in f:
        a = line.split('\'')
        type = a[1]
        indicators = a[2].split()
        type_indicators[type] = indicators
save_indicators = {}
with open('data/type_indicators/save_indicator.txt', 'r') as f:
    for line in f:
        a = line.split('\'')
        type = a[1]
        indicators = a[2].split()
        save_indicators[type] = indicators

In [3]:
# naive functions for type from text keywords

def filtering(text):
    pred = []
    for type in types:            
        if type in text:
            pred.append(type)
    if ('drawing' in text) or ('sketch' in text) or ('design' in text):
        pred.append('design/drawing/sketch')
    if len(pred) > 0:
        return pred[-1]
    else:
        return -1
    
def indicating(text):
    pred = []
    for type in types:
        for indicator in type_indicators[type]:
            if indicator in text:
                pred.append(type)
    if len(pred) > 0:
        return pred[-1]
    else:
        return -1

def save_indicating(text):
    pred = []
    for type in types:
        if type in save_indicators.keys():
            for indicator in save_indicators[type]:
                if indicator in text:
                    pred.append(type)
    if len(pred) > 0:
        return pred[-1]
    else:
        return -1


In [4]:
# convert to numeric
def replace_value(value: str):
    if pd.isnull(value):
        return value
    return np.float64(value.replace(',', '.'))


# convert to numeric and only keep year part
def replace_start_end(value: str):
    if pd.isnull(value):
        return value
    if re.match('^d?ddd$', value):
        return int(value)
    if re.match('dddd$', value):
        return int(value[-4:])
    elif not value[0].isdigit():
        return int(f'19{value[-2:]}')
    else:
        return nan


def extract_year_from_name(row):
    name = row['name']
    start = row['start']
    if pd.isnull(start) and not pd.isnull(name):
        match = re.search('\d\d\d\d', name)
        if match:
            start = match.group()
    return start


def preprocess_dataframe(df, submission=False):
    categorical_cols = ['material', 'location', 'before_Christ', 'country_and_unit', 'technique', 'parameter',
                        'museum_abbr', 'damages', 'state', 'color', 'event_type', 'collection_mark']
    categorical_cols += ['unit', 'participants_role', 'participant', 'musealia_mark']

    # just keeping track what values are used
    numeric_cols = ['start', 'end', 'value', 'collection_queue_nr', 'is_original', 'ks', 'element_count',
                    'musealia_seria_nr', 'musealia_queue_nr']

    dropped_cols = ['id', 'parish']  # can't use
    dropped_cols += ['full_nr', 'class', 'collection_additional_nr', 'additional_text', 'text', 'initial_info',
                     'musealia_additional_nr']  # 'commentary','name', 'legend'

    if not submission: dropped_cols.append('type')

    df['start'] = df['start'].apply(replace_start_end)
    df['end'] = df['end'].apply(replace_start_end)
    df['value'] = df['value'].apply(replace_value)
    df['start'] = df[['name', 'start']].apply(extract_year_from_name, axis=1)

    df = df.drop(columns=dropped_cols)
    df = pd.get_dummies(df, columns=categorical_cols)
    df = df.fillna(0)
    return df

In [5]:
def extract_label_from_comment(row):
    # comment #################################################
    comment = row['commentary']

    if not pd.isnull(comment):
        comment = str(comment).lower()

        comment_dict = {
            'lakk': 'pitser/templijäljend',
            'must-valge negatiiv': 'fotonegatiiv',
            'pitserilakk': 'pitser/templijäljend',
            'käepide': 'pitsat',
            'перф': 'fotonegatiiv',
            'fotoemulsioon': 'fotomaterjal',
            'plakat':'plakat'
        }
        for key, val in comment_dict.items():
            if comment.startswith(key):
                return val

        if re.match('^\d,\d\d\sg$', comment):
            return 'münt'

        if 'diapositiiv' in comment:
            return 'diapositiiv'

    # name #################################################
    name = row['name']

    if not pd.isnull(name):
        name = str(name).lower()
        if name == ['denaar', 'killing', 'penn', 'schilling', '1/2 örtug', 'dirhem', 'fyrk']:
            return 'münt'

        for val in ['medal', 'plakat', 'märkmed', 'maal', 'kiri', 'kleit', 'kava', 'joonistus', 'graafika', 'dokument',
                    'ajakiri', 'telegramm', 'skulptuur', 'raamat', 'postkaart', 'nukk', 'skulptuur', 'käsikiri']:
            if name.startswith(val):
                return val

        name_dict = {
            'kaustik': 'kaustik/vihik',
            'vihik': 'kaustik/vihik',
            'reprofoto': 'diapositiiv',
        }
        for key, val in name_dict.items():
            if name.startswith(key):
                return val
    return nan

def replace_predictions(labels, pred):
    result = np.array(pred, copy=True)
    for i, label in enumerate(labels):
        if not pd.isnull(label) and label != 0:
            result[i] = label
    return result

# combine models via class-probability combination (soft-voting)


In [6]:
# is the full ds used for submission?
full = False
# submit to 
sub_name = 'kaspar_type_checks_xgrfnn_no_emb.csv'

In [7]:
#define models to be used for testing use 03 for submission use full
import pickle
xgb = XGBClassifier()
xgb.load_model('models/xg/xg_est_data_smote100_03.json')

rf = pickle.load(open('./models/rf/train_prep_full_best' , 'rb'))

boost_emb = XGBClassifier()
boost_emb.load_model('models/nlp/xgboost_03.json')

nn = TabNetClassifier()
nn.load_model('models/nn/tabnet_full.zip')

/home/till/.local/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


In [8]:
data = test_est_prepared.copy() if full else val_est_prepared.copy()

features = data.drop('type', axis=1)
labels = data.type

if not full:
    # at least xgboost cannot deal with string labels
    label_encoder = LabelEncoder()
    label_encoder = label_encoder.fit(labels)
    labels = label_encoder.transform(labels)
    y_test = labels

X_test = features

In [9]:
results = pd.DataFrame()
results['id'] = X_test.index
results.set_index('id', inplace=True)
if not full: results['type'] = y_test

#results['rf'] = rf.predict(X_test)
#results['xg'] = xgb.predict_proba(X_test)
#results['nn'] = nn.predict(X_test.values)

results['filter'] = [-1] * len(results)
results['indi'] = [-1] * len(results)
results['save'] = [-1] * len(results)
results['emb'] = [[-1]] * len(results)


results['xg'] = [[-1]] * len(results)

In [10]:
for i,item in enumerate(xgb.predict_proba(X_test)):
    results['xg'].iloc[i] = np.array(item)

/tmp/ipykernel_16608/1290882250.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['xg'].iloc[i] = np.array(item)


In [11]:
text = test_curie.copy() if full else val_curie.copy()

features = text.drop('type', axis=1)
labels = text.type

#text['pred'] = boost_emb.predict_proba(features)
text['pred'] = [[-1]] * len(features)

In [12]:
for i,item in enumerate(boost_emb.predict_proba(features)):
    text['pred'].iloc[i] = np.array(item)

/tmp/ipykernel_16608/2524059003.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  text['pred'].iloc[i] = np.array(item)


In [13]:
for index, item in text.iterrows():
        if index in results.index:
            results.at[index, 'emb'] = item['pred']

# evalaluate

In [14]:
from operator import add
def vote(preds):
    if preds[-1][0] == -1:
        preds = preds[:-1]
    res = np.sum(preds, axis=0)
    return np.argmax(res)

In [23]:
train, val = train_test_split(train_prep, test_size=0.3, random_state=0)

In [25]:
train.to_csv('data/prepared_ready/train_prepared.csv')
val.to_csv('data/prepared_ready/val_prepared.csv')

In [15]:
results['prediction'] = results.apply(lambda row: vote([row.xg, row.emb]), axis=1)
if not full:
    results.prediction = results.prediction.replace(type_lookup.id.to_list(), type_lookup.estonian.to_list())

    a = results.prediction.copy().tolist()

    df_submission = pd.read_csv("data/general/val.csv")
    x2 = preprocess_dataframe(df_submission, submission=True)
    # reorder columns + add missing columns + remove extra columns
    x2_labels = x2.apply(extract_label_from_comment, axis=1)    

    results.prediction = replace_predictions(x2_labels, results.prediction)
    submission = pd.DataFrame({'id': results.index ,'type': results.prediction})

    b = results.prediction.copy().tolist()

    count = 0
    for i in range(len(a)):
        if a[i] != b[i]:
            count += 1
            print(a[i], b[i])

    print(count)

    results.type = results.type.replace(type_lookup.id.to_list(), type_lookup.estonian.to_list())

    print(accuracy_score(results.type, results.prediction))
    print(classification_report(results.type, results.prediction))

kaustik/vihik märkmed
kiri, postkaart kiri
kiri postkaart
fotonegatiiv kiri
dokument kava
foto diapositiiv
foto skulptuur
väiketrükis postkaart
kavand/joonis/eskiis kava
graafika joonistus
10
0.960952380952381
                               precision    recall  f1-score   support

                      ajakiri       1.00      0.88      0.94        33
                      ajaleht       0.83      0.74      0.78        34
                        album       1.00      0.85      0.92        13
          arheoloogiline leid       1.00      1.00      1.00       240
             aukiri/auaadress       1.00      0.70      0.82        10
                  diapositiiv       0.95      1.00      0.97        19
           digitaalne kujutis       1.00      1.00      1.00        57
                     dokument       0.83      0.89      0.86       125
                          ehe       1.00      1.00      1.00         6
                         foto       0.97      0.99      0.98      1067
        

In [21]:
results[results.type == 'aukiri/auaadress']

,type,filter,indi,save,emb,xg,prediction
id,,,,,,,
3981017,aukiri/auaadress,-1,-1,-1,"[0.05482011, 0.0065371706, 0.004327592, 0.0033103472, 0.0057972725, 0.0033087928, 0.005312763, 0.0041445396, 0.0039782496, 0.004369275, 0.003493294, 0.0028150713, 0.0029248407, 0.010927359, 0.11970181, 0.0044690203, 0.009431428, 0.003429786, 0.011684738, 0.0065783095, 0.0075291474, 0.006122921, 0.04034316, 0.09955946, 0.009296848, 0.016754968, 0.016625019, 0.0038950744, 0.007263255, 0.0056075403, 0.0027574417, 0.010583477, 0.06880006, 0.0048672473, 0.0031847265, 0.007827149, 0.25767124, 0.035119977, 0.0044708448, 0.0035428074, 0.004447958, 0.001981915, 0.005400954, 0.015822336, 0.0066100056, 0.0028548504, 0.0038920257, 0.006920653, 0.03261955, 0.003431533, 0.002740225, 0.011506938, 0.003415175, 0.009222703, 0.0059490027]","[3.7493588e-05, 3.3211967e-05, 6.718247e-05, 2.148774e-05, 0.00027233292, 7.523039e-05, 2.1203827e-05, 3.3023665e-05, 5.0521943e-05, 1.9591118e-05, 2.0848433e-05, 8.974229e-05, 2.6517198e-05, 3.5062236e-05, 0.001228506, 6.0772723e-05, 1.6292895e-05, 1.6710923e-05, 3.0420912e-05, 0.58532476, 1.6482089e-05, 1.3070626e-05, 0.01896723, 0.07556466, 2.5793084e-05, 0.0006818629, 0.00018835215, 5.852108e-05, 5.6444725e-05, 3.2559776e-05, 8.168336e-06, 4.733427e-05, 0.00022884808, 9.5231546e-05, 3.645736e-05, 0.00017868419, 0.02611204, 4.6379937e-05, 8.92714e-06, 0.00026367634, 3.4266945e-05, 0.00363886, 0.027102204, 7.411641e-05, 3.1205407e-05, 4.566685e-05, 2.515986e-05, 3.3118726e-05, 3.0267278e-05, 2.6893458e-05, 4.527091e-05, 0.00028714054, 1.8289616e-05, 1.8518875e-05, 0.25847742]",kutse
3919333,aukiri/auaadress,-1,-1,-1,"[5.455613e-05, 2.9018593e-05, 5.1901276e-05, 4.953952e-05, 3.3625485e-05, 8.44691e-05, 6.3002226e-05, 6.5141576e-05, 4.5150573e-05, 4.6654408e-05, 4.7051093e-05, 2.1001213e-05, 4.05426e-05, 5.8504793e-05, 0.002722113, 6.391697e-05, 0.000100076686, 8.698379e-05, 4.4668537e-05, 0.0001707827, 4.2809635e-05, 0.0006295877, 0.006583444, 0.9856528, 7.549475e-05, 3.7460493e-05, 0.00021332045, 6.805574e-05, 6.4047854e-05, 0.00024764065, 4.93309e-05, 3.8167185e-05, 6.676827e-05, 9.604127e-05, 4.7659592e-05, 5.1244144e-05, 0.0002871228, 0.0002792484, 8.567215e-05, 5.616535e-05, 5.9146805e-05, 0.00025728907, 0.00018804094, 3.814415e-05, 8.854093e-05, 5.6457222e-05, 7.186659e-05, 0.00013256414, 0.00012873393, 5.1353043e-05, 4.3712716e-05, 9.2473834e-05, 5.110827e-05, 7.1364666e-05, 0.00011843657]","[3.3368382e-05, 3.24704e-05, 6.5682405e-05, 2.1007958e-05, 8.6437e-05, 6.602499e-05, 1.530377e-05, 7.919516e-05, 4.9393875e-05, 2.3682496e-05, 2.0382924e-05, 2.282999e-05, 2.7821097e-05, 3.4848315e-05, 0.0077552563, 5.941578e-05, 0.00022608397, 8.21487e-06, 4.588319e-05, 0.008883088, 1.1483538e-05, 1.8231589e-05, 0.68556213, 0.25389504, 2.2570512e-05, 5.6596775e-05, 0.0009525746, 5.479174e-05, 5.5184366e-05, 3.1832773e-05, 8.21545e-06, 6.024293e-05, 0.00013472026, 0.0021103963, 3.564333e-05, 3.6529153e-05, 0.00043715938, 4.5344354e-05, 8.006517e-06, 4.609116e-05, 3.3501794e-05, 0.003468853, 0.03243137, 5.3845597e-05, 3.076855e-05, 1.869431e-05, 3.5977537e-05, 3.237921e-05, 3.206993e-05, 2.629295e-05, 4.4260087e-05, 0.0021687571, 1.7881224e-05, 1.9134859e-05, 0.00044701443]",aukiri/auaadress
3357383,aukiri/auaadress,-1,-1,-1,"[0.0002968234, 0.000104709405, 0.00022225745, 0.0001787915, 0.0003193982, 0.00039727162, 0.00020423828, 0.00014923282, 0.00034095583, 0.00024111076, 0.00033520145, 9.626588e-05, 0.00021201523, 0.0002519607, 0.00061706104, 0.00024390333, 0.00013444416, 0.00021485344, 6.000971e-05, 0.00024743256, 0.00018960515, 0.0005009036, 0.00049251295, 0.97482705, 0.00023598812, 0.0004114674, 0.0003035261, 0.00023660716, 0.0030597365, 0.00027730115, 0.00031913817, 0.0006262896, 0.0009564463, 0.00019306986, 0.00025390126, 0.0003521886, 0.004736345, 0.00057706016, 0.00013655965, 0.00024737947, 0.0002440867, 0.0006974296, 0.00013084496, 0.00045648063, 0.00022526628, 0.00036406322, 0.00014053729, 0.00043958865, 0.00021325586,

In [ ]:
album foto
ajaleht dokument


# submission

In [16]:
if full:
    results.prediction = results.prediction.replace(type_lookup.id.to_list(), type_lookup.estonian.to_list())

    a = results.prediction.copy().tolist()

    df_submission = pd.read_csv("data/general/test.csv")
    x2 = preprocess_dataframe(df_submission, submission=True)
    # reorder columns + add missing columns + remove extra columns
    x2_labels = x2.apply(extract_label_from_comment, axis=1)    

    results.prediction = replace_predictions(x2_labels, results.prediction)
    submission = pd.DataFrame({'id': results.index ,'type': results.prediction})

    b = results.prediction.copy().tolist()

    count = 0
    for i in range(len(a)):
        if a[i] != b[i]:
            count += 1
            print(a[i], b[i])

    print(count)


    #submission.groupby('type').nunique()  # predicted classes

    submission.to_csv(f'submissions/{sub_name}', index=False)